In [1]:
import os
import numpy as np
from PIL import Image
import torch
from pycocotools.coco import COCO
from torch.utils.data import Dataset
import json

In [2]:
# ============================================================
# 1) MONTAR DRIVE E DESCOMPACTAR O DATASET
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

zip_path = "/content/drive/MyDrive/dataset_final_coco.zip"

import zipfile
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall('/content/dataset')

root_dir = "/content/dataset/dataset_final_coco"
print("Dataset extraído em:", root_dir)


Mounted at /content/drive
Dataset extraído em: /content/dataset/dataset_final_coco


In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ------------------------------
# TRAIN TRANSFORMS (fortes)
# ------------------------------
def get_train_transforms():
    return A.Compose(
        [
            A.HueSaturationValue(
                hue_shift_limit=0.015,
                sat_shift_limit=0.7,
                val_shift_limit=0.4,
                p=1.0
            ),

            A.Rotate(limit=30, border_mode=0, p=1.0),

            A.Affine(
                translate_percent=0.1,
                scale=(0.5, 1.5),
                shear=(-5, 5),
                p=1.0
            ),

            A.Perspective(scale=(0.0005, 0.0005), p=1.0),

            A.HorizontalFlip(p=0.5),

            A.RandomCrop(height=512, width=512, p=1.0, pad_if_needed=True),
            A.Resize(height=640, width=640, p=1.0),

            A.Normalize(
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                max_pixel_value=255.0
            ),

            ToTensorV2()
        ],
        bbox_params=A.BboxParams(
            format="pascal_voc",
            label_fields=["labels"],
            min_visibility=0.2
        )
    )


# ------------------------------
# VAL/TEST TRANSFORMS (leves)
# ------------------------------
def get_eval_transforms():
    return A.Compose(
        [
            A.Resize(640, 640),
            A.Normalize(
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                max_pixel_value=255.0
            ),
            ToTensorV2()
        ],
        bbox_params=A.BboxParams(
            format="pascal_voc",
            label_fields=["labels"],
            min_visibility=0.0
        )
    )


In [4]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import torch
import os
from pycocotools.coco import COCO

class CocoAlbumentationsDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, ann_file, transforms=None):
        import json
        self.img_dir = img_dir
        self.transforms = transforms

        with open(ann_file, "r") as f:
            data = json.load(f)

        self.images = {img["id"]: img for img in data["images"]}

        # agrupar anotações por imagem
        self.annotations = {}
        for ann in data["annotations"]:
            img_id = ann["image_id"]
            if img_id not in self.annotations:
                self.annotations[img_id] = []
            self.annotations[img_id].append(ann)

        self.ids = list(self.images.keys())

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        info = self.images[img_id]

        img_path = os.path.join(self.img_dir, info["file_name"])
        image = np.array(Image.open(img_path).convert("RGB"))

        annots = self.annotations.get(img_id, [])

        # se não tem boxes → pula imagem
        if len(annots) == 0:
            return None

        boxes = []
        labels = []

        for ann in annots:
            x, y, w, h = ann["bbox"]
            boxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"])

        if self.transforms:
            transformed = self.transforms(
                image=image,
                bboxes=boxes,
                labels=labels
            )

            image = transformed["image"]
            boxes = transformed["bboxes"]
            labels = transformed["labels"]

        # 🚨 Proteção contra augmentations removerem TODAS as caixas
        if len(boxes) == 0:
            return None

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels
        }

        return image, target

    def __len__(self):
        return len(self.ids)



In [5]:
def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    return tuple(zip(*batch))


In [6]:
train_dataset = CocoAlbumentationsDataset(
    f"{root_dir}/train/images",
    f"{root_dir}/coco_annotations_train.json",
    transforms=get_train_transforms()
)

val_dataset = CocoAlbumentationsDataset(
    f"{root_dir}/val/images",
    f"{root_dir}/coco_annotations_val.json",
    transforms=get_eval_transforms()
)

test_dataset = CocoAlbumentationsDataset(
    f"{root_dir}/test/images",
    f"{root_dir}/coco_annotations_test.json",
    transforms=get_eval_transforms()
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    collate_fn=collate_fn
)


In [7]:
import torchvision
import torch.nn as nn

num_classes = 2  # background + gun

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    weights="DEFAULT",
    box_score_thresh=0.5,
)

# substituir head com dropout
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
    in_features, num_classes
)

# aplicar dropout manualmente
model.roi_heads.box_predictor.cls_score = nn.Sequential(
    nn.Dropout(0.05),
    nn.Linear(in_features, num_classes)
)

model = model.cuda()

# Otimizador
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.01,
    momentum=0.937,
    weight_decay=0.0005
)

lr_scheduler = torch.optim.lr_scheduler.LinearLR(
    optimizer,
    start_factor=1.0,
    end_factor=0.01,
    total_iters=100  # 100 épocas
)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:00<00:00, 184MB/s]


In [8]:
import os
import json

save_dir = "runs/fasterrcnn/train1"
os.makedirs(save_dir, exist_ok=True)

history = {
    "train_loss": [],
    "lr": [],
    "epoch": [],
    "map50": [],
    "map5095": []
}

with open(f"{save_dir}/history.json", "w") as f:
    json.dump(history, f, indent=4)


In [9]:
# ================================================================
# MÉTRICAS COCO COMPLETAS
# ================================================================

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch


def evaluate_coco(model, dataset, ann_path, device="cuda"):
    model.eval()

    coco_gt = COCO(ann_path)
    results = []

    for img, target in dataset:
        img_tensor = img.to(device).unsqueeze(0)

        with torch.no_grad():
            output = model(img_tensor)[0]

        boxes = output["boxes"].cpu().numpy()
        scores = output["scores"].cpu().numpy()
        labels = output["labels"].cpu().numpy()

        image_id = int(target["image_id"].item())

        for box, score, label in zip(boxes, scores, labels):
            x1, y1, x2, y2 = box.tolist()
            results.append({
                "image_id": image_id,
                "category_id": 1,
                "bbox": [x1, y1, x2 - x1, y2 - y1],
                "score": float(score)
            })

    coco_dt = coco_gt.loadRes(results)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # mAP50 = COCO metric 1
    # mAP50-95 = COCO metric 0
    map50 = coco_eval.stats[1]
    map5095 = coco_eval.stats[0]

    return map50, map5095


# ================================================================
# FUNCTION — CONFUSION MATRIX
# ================================================================
def plot_confusion_matrix(y_true, y_pred, save_path):
    labels = ["background", "Gun"]
    cm = np.zeros((2, 2), dtype=int)

    for t, p in zip(y_true, y_pred):
        cm[t][p] += 1

    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=labels, yticklabels=labels)
    plt.title("Confusion Matrix")
    plt.savefig(save_path)
    plt.close()


# ================================================================
# FUNCTION — PRECISION-RECALL CURVE
# ================================================================
def plot_pr_curve(precisions, recalls, save_path):
    plt.figure(figsize=(7,5))
    plt.plot(recalls, precisions)
    plt.title("Precision-Recall Curve")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.grid()
    plt.savefig(save_path)
    plt.close()


# ================================================================
# FUNCTION — F1 CURVE
# ================================================================
def plot_f1_curve(precisions, recalls, save_path):
    f1 = 2 * (precisions * recalls) / (precisions + recalls + 1e-6)
    plt.figure(figsize=(7,5))
    plt.plot(recalls, f1)
    plt.title("F1 Curve")
    plt.xlabel("Recall")
    plt.ylabel("F1-score")
    plt.grid()
    plt.savefig(save_path)
    plt.close()


In [ ]:
import matplotlib.pyplot as plt

best_loss = float("inf")
patience = 20
counter = 0

for epoch in range(100):
    model.train()
    total_loss = 0

    for batch_idx, (imgs, targets) in enumerate(train_loader):

        # 🔍 LOG DO BATCH
        print(f"[Epoch {epoch}] Batch {batch_idx}: {len(imgs)} imagens")

        # detectar erros comuns
        for i, t in enumerate(targets):
            if t["boxes"].shape[0] == 0:
                print(f"  ⚠️  Atenção: imagem {i} no batch {batch_idx} está com boxes vazios!")

        # GPU
        imgs = [img.cuda() for img in imgs]
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]

        # forward
        loss_dict = model(imgs, targets)
        losses = sum(loss_dict.values())

        # outra linha de LOG opcional:
        print(f"  Loss do batch: {losses.item():.4f}")

        # backward
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()

    lr_scheduler.step()

    print(f"[Epoch {epoch}] Loss: {total_loss:.4f}")

    # =======================================================
    # AVALIAÇÃO COCO EM VAL
    # =======================================================
    map50, map5095 = evaluate_coco(
        model,
        val_dataset,
        f"{root_dir}/annotations/instances_val.json"
    )

    print(f"   mAP50={map50:.4f} | mAP50-95={map5095:.4f}")

    # =======================================================
    # Salvar histórico
    # =======================================================
    history["train_loss"].append(total_loss)
    history["map50"].append(map50)
    history["map5095"].append(map5095)
    history["lr"].append(optimizer.param_groups[0]["lr"])
    history["epoch"].append(epoch)

    with open(f"{save_dir}/history.json", "w") as f:
        json.dump(history, f, indent=4)

    # =======================================================
    # EARLY STOPPING
    # =======================================================
    if total_loss < best_loss:
        best_loss = total_loss
        counter = 0
        torch.save(model.state_dict(), f"{save_dir}/best_model.pth")
    else:
        counter += 1
        if counter >= patience:
            print("EARLY STOPPING")
            break




# =======================================================
# GERAR GRÁFICOS APÓS O TREINAMENTO
# =======================================================

# Loss plot
plt.figure()
plt.plot(history["epoch"], history["train_loss"])
plt.title("Training Loss")
plt.savefig(f"{save_dir}/losses.png")
plt.close()

# mAP plot
plt.figure()
plt.plot(history["epoch"], history["map50"], label="mAP50")
plt.plot(history["epoch"], history["map5095"], label="mAP50-95")
plt.legend()
plt.title("mAP Curve")
plt.savefig(f"{save_dir}/map_curve.png")
plt.close()


[Epoch 0] Batch 0: 16 imagens
  Loss do batch: 1.3621
[Epoch 0] Batch 1: 15 imagens
  Loss do batch: 0.4601
[Epoch 0] Batch 2: 15 imagens
  Loss do batch: 0.4506
[Epoch 0] Batch 3: 13 imagens
  Loss do batch: 0.4326
[Epoch 0] Batch 4: 13 imagens
  Loss do batch: 0.3895
[Epoch 0] Batch 5: 15 imagens
  Loss do batch: 0.5037
[Epoch 0] Batch 6: 14 imagens
  Loss do batch: 0.6253
[Epoch 0] Batch 7: 15 imagens
  Loss do batch: 0.2614
[Epoch 0] Batch 8: 15 imagens
  Loss do batch: 0.3458
[Epoch 0] Batch 9: 14 imagens
  Loss do batch: 0.6919
[Epoch 0] Batch 10: 16 imagens
  Loss do batch: 0.2953
[Epoch 0] Batch 11: 15 imagens
  Loss do batch: 0.4064
[Epoch 0] Batch 12: 15 imagens
  Loss do batch: 0.2529
[Epoch 0] Batch 13: 16 imagens
  Loss do batch: 0.2422
[Epoch 0] Batch 14: 16 imagens
  Loss do batch: 0.2469
[Epoch 0] Batch 15: 14 imagens
  Loss do batch: 0.2342
[Epoch 0] Batch 16: 15 imagens
  Loss do batch: 0.2719
[Epoch 0] Batch 17: 16 imagens
  Loss do batch: 0.2893
[Epoch 0] Batch 18: 

In [ ]:
# ================================================================
# FINAL EVALUATION FOR PR + F1 + CONFUSION MATRIX
# ================================================================

all_scores = []
all_labels = []
all_preds = []

model.eval()
for img, target in val_dataset:
    img_tensor = img.cuda().unsqueeze(0)

    with torch.no_grad():
        out = model(img_tensor)[0]

    scores = out["scores"].cpu().numpy()
    labels_pred = out["labels"].cpu().numpy()
    labels_true = target["labels"].numpy()

    # armazenar
    all_scores.extend(scores)
    all_preds.extend(labels_pred)
    all_labels.extend(labels_true)


# ORDENAR POR SCORE PARA PR CURVE
order = np.argsort(-np.array(all_scores))
preds = np.array(all_preds)[order]
labels = np.array(all_labels)[order]

tp = (preds == labels)
fp = (preds != labels)
fn = (labels != preds)

precision = np.cumsum(tp) / (np.cumsum(tp + fp) + 1e-6)
recall = np.cumsum(tp) / (len(labels) + 1e-6)

plot_pr_curve(precision, recall, f"{save_dir}/pr_curve.png")
plot_f1_curve(precision, recall, f"{save_dir}/f1_curve.png")
plot_confusion_matrix(all_labels, all_preds, f"{save_dir}/confusion_matrix.png")


In [ ]:
from PIL import Image

img1 = Image.open(f"{save_dir}/losses.png")
img2 = Image.open(f"{save_dir}/map_curve.png")
img3 = Image.open(f"{save_dir}/pr_curve.png")
img4 = Image.open(f"{save_dir}/f1_curve.png")

width = max(img1.width, img2.width)
height = img1.height + img2.height + img3.height + img4.height

results = Image.new("RGB", (width, height), "white")

y = 0
for img in [img1, img2, img3, img4]:
    results.paste(img, (0, y))
    y += img.height

results.save(f"{save_dir}/results.png")


In [ ]:
model.load_state_dict(torch.load("best_fasterrcnn.pth"))
model.eval()

results = []

for img, target in test_dataset:
    img_tensor = img.cuda().unsqueeze(0)

    with torch.no_grad():
        output = model(img_tensor)[0]

    results.append({
        "image_id": target["image_id"].item(),
        "boxes": output["boxes"].cpu().tolist(),
        "scores": output["scores"].cpu().tolist(),
        "labels": output["labels"].cpu().tolist()
    })

print("Inferência concluída!")
